In [1]:
import string
import time, copy 
import pandas as pd
import yfinance as yf 
import seaborn as sns
from joblib import dump,load
import matplotlib.pyplot as plt
from IPython.display import clear_output
from datetime import datetime, timedelta,date
import tweepy, re, io, csv, os, pymongo, numpy as np
%matplotlib inline
#sentiment
import nltk
import mglearn
from nltk.corpus import stopwords
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import confusion_matrix, classification_report, roc_curve
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer,TfidfTransformer
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from tensorflow.keras.utils import to_categorical
#forecasting component 
import os
import time
import warnings
import numpy as np
from numpy import newaxis
import tensorflow as tf
physical_devices = tf.config.list_physical_devices('GPU')
try:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
    print("Growth SET")
except:
    print("Invalid device OR cannot modify virtual devices once initialized.")
    pass
from tensorflow import keras
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.layers import LSTM
from tensorflow.keras import layers
from tensorflow.keras.layers import *
from tensorflow.keras.models import Sequential
from tensorflow.keras.activations import *
from tensorflow.keras.callbacks import EarlyStopping 
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from sklearn.metrics import mean_squared_error
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

plt.rcParams["figure.figsize"] = [20, 5]
print( "GPU ACTIVE" if len(tf.config.list_physical_devices('GPU'))>0 else "GPU NOT AVAILABLE")

Growth SET
GPU ACTIVE


In [7]:
auth = tweepy.OAuthHandler('9tpKa7Rs0azYhUUZQ9t13zZbR', 'NJvdMLo901KiycSjlz1NlshQaM8iJmRLfHCXZhZXgUzwy2cl4y')
auth.set_access_token("1070877312604954625-91M4PyNC72QYws3lIm90mj5yIjElVU", "EpARpeIQECJDEWRe2WfWg239FQaHoWZa4xFreDMTm84Zf")
# auth = tweepy.OAuthHandler('qu53XjVh2PxvXPTeYfCaW5Ujb', 'PYtXz1VB0B9AhDusAzreRnVZso9hfeSUyA0jAhpQqxDtpziM3e')
# auth.set_access_token("1070877312604954625-h2OYl24cl3a01CYM2QHuZIG2ykEvTy", "1ZoLMdm663zDbuNGDdmx1OhEJKgjVYI7WOOmwnfCITbFV")
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

In [8]:
myclient = pymongo.MongoClient("mongodb://localhost:27017/")
DBlist=myclient.list_database_names()
data = api.rate_limit_status()
print (data['resources']['statuses']['/statuses/home_timeline'])
print (data['resources']['users']['/users/lookup'])
mydb = None
if "sentimentDB" in DBlist:
    print("sentimentDB exists.")
    mydb = myclient["sentimentDB"]
else:
    print("sentimentDB Didn't Exist and Has Been Created\n")
    mydb = myclient["sentimentDB"]
print(mydb)
processed = mydb["Processed"]
sentiment140 = mydb["sentiment140"]
unprocessed = mydb["Unprocessed"]
unprocessedArchive = mydb["Unprocessed Archive"]

{'limit': 15, 'remaining': 15, 'reset': 1616024585}
{'limit': 900, 'remaining': 900, 'reset': 1616024585}
sentimentDB exists.
Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'sentimentDB')


In [4]:
def printTweet(tweet): 
    print("T: " + str(tweet.created_at)) 
    # print("The id is : " + str(tweet.id)) 
    # print("The id_str is : " + tweet.id_str) 
#   # print(tweet.extended_tweet)
    if hasattr(tweet, "retweeted_status"):  # Check if Retweet
        try:
            print(tweet.retweeted_status.extended_tweet["full_text"])
        except AttributeError:
            print(tweet.retweeted_status.text)
    else:
        try:
            print(tweet.extended_tweet["full_text"])
        except AttributeError:
            print(tweet.full_text)
    print("Retweet Count:\t{0}".format(int(tweet.retweet_count)))
    # print("The entitities are : " + str(tweet.entities)) 
    # print("The source is : " + tweet.source) 
    # print("The source_url is : " + tweet.source_url)   
    # print("The in_reply_to_status_id is : " + str(tweet.in_reply_to_status_id)) 
    # print("The in_reply_to_status_id_str is : " + str(tweet.in_reply_to_status_id_str)) 
    # print("The in_reply_to_user_id is : " + str(tweet.in_reply_to_user_id)) 
    # print("The in_reply_to_user_id_str is : " + str(tweet.in_reply_to_user_id_str)) 
    # print("The in_reply_to_screen_name is : " + str(tweet.in_reply_to_screen_name)) 
    print("The poster's screen name is : " + tweet.user.screen_name) 
    print("The poster's name is : " + tweet.user.name) 
    # print("The geo is : " + str(tweet.geo)) 
    # print("The coordinates are : " + str(tweet.coordinates)) 
    # print("The place is : " + str(tweet.place)) 
    print("The contributors are : " + str(tweet.contributors)) 
    print("The is_quote_status is : " + str(tweet.is_quote_status)) 
    print("The retweet_count is : " + str(tweet.retweet_count)) 
    print("The favorite_count is : " + str(tweet.favorite_count)) 
    # print("Has the authenticated user favourited the status? : " + str(tweet.favorited)) 
    # print("Has the authenticated user retweeted the status? " + str(tweet.retweeted)) 
    # try:
    #     print("Is the status possibly_sensitive? : " + str(tweet.possibly_sensitive)) 
    # except: 
    #     tweet.possibly_sensitive="Unknown"
    #     print("Is the status possibly_sensitive? : " + str(tweet.possibly_sensitive)) 
    # print("The lang is : " + tweet.lang) 


In [1]:
def insterDFToDB(column, dataframe):
    return mydb[column].insert_many(dataframe.to_dict(orient='records'))

In [5]:
def insertToDB(tweet, stockInfo,query):
    t=datetime.utcnow()  
    tweetText="String Error Did Not Find String."
    sensitive=None
    if hasattr(tweet, "retweeted_status"):  # Check if Retweet
        try:
            tweetText=(tweet.retweeted_status.extended_tweet["full_text"])
        except AttributeError:
            tweetText=(tweet.retweeted_status.text)
    else:
        try:
            tweetText=(tweet.extended_tweet["full_text"])
        except AttributeError:
            try:
                tweetText=(tweet.full_text)
            except AttributeError:
                tweetText=(tweet.text)
    try:
        sensitive=str(tweet.possibly_sensitive)
    except: 
        tweet.possibly_sensitive="Unknown"
    
    item= { "dateRetrieved"    :t,
            "dateCreated"      :t,
            "interval(S)"      :(t-tweet.created_at).total_seconds(),
            "stockInfo"        :stockInfo,
            "query"            :query,
            "_id"              :str(tweet.id_str),
            "poster"           :str(tweet.user.screen_name),
            "posterID"         :str(tweet.user.id_str),
            "posterVerified"   :tweet.user.verified,
            "followers"        :int(tweet.user.followers_count),
            "friends"          :int(tweet.user.friends_count),
            "likes"            :int(tweet.favorite_count),
            "retweets"         :int(tweet.retweet_count),
#             "quotes"         :int(tweet.quote_count),
            "tweetText"        :tweetText,
            "sensitive"        :sensitive,
            "isRetweet"        :False,
            "isQuote"          :False
          }
    try:
        unprocessed.insert_one(item)
        return False
    except pymongo.errors.DuplicateKeyError:
#         print("duplicate entry found")
        unprocessed.replace_one({'_id': tweet.id_str}, item, upsert=True)
        return True

In [6]:
def SearchStore(query,stockinfo ,sinceDate=str(datetime.utcnow() - timedelta(days=30))[:10],
                        untilDate=str(datetime.utcnow())[:10],
                        sID=None, rtmin=2,count=10000):
    counter=0
    updateC=0
    newEntries=0
    for status in tweepy.Cursor(api.search, include_rts =False, include_replies=False, 
                                since=sinceDate, until=untilDate,tweet_mode="extended"
                                ,q=query+" exclude:retweets AND exclude:replies AND min_retweets:"+str(rtmin), lang="en").items():
        # process status here
        if(insertToDB(status,stockinfo,query)):updateC+=1            
        else:newEntries+=1
        counter+=1
        if counter>=count:
            break
    if counter!=count: print("End Reached")
    print(stockinfo+"\tEntries: {0}\tUpdated: {1}\tNew: {2}".format(counter,updateC,newEntries))    

In [7]:
def ArchiveStore(q, sinceDate, untilDate,stockInfo="", limit=100):
    t=datetime.utcnow()
    count=0
    for tweet in tweepy.Cursor(api.search_full_archive,environment_name ='Sentiment', query =q+" lang:en",
                                fromDate=sinceDate, toDate=untilDate).items():
        tweetText="String Error Did Not Find String."
        sensitive=None
        if hasattr(tweet, "retweeted_status"):
            try:
                tweetText=(tweet.retweeted_status.extended_tweet["full_text"])
            except AttributeError:
                tweetText=(tweet.retweeted_status.text)
        else:
            try:
                tweetText=(tweet.extended_tweet["full_text"])
            except AttributeError:
                try:
                    tweetText=(tweet.full_text)
                except AttributeError:
                    tweetText=(tweet.text)
        try:
            sensitive=str(tweet.possibly_sensitive)
        except: 
            tweet.possibly_sensitive="Unknown"
        count+=1
        item= {     "dateRetrieved"    :t,
                    "dateCreated"      :tweet.created_at,
                    "interval(S)"      :(t-tweet.created_at).total_seconds(),
                    "stockInfo"        :stockInfo,
                    "query"            :q,
                    "_id"              :str(tweet.id_str),
                    "poster"           :str(tweet.user.screen_name),
                    "posterID"         :str(tweet.user.id_str),
                    "posterVerified"   :tweet.user.verified,
                    "followers"        :int(tweet.user.followers_count),
                    "friends"          :int(tweet.user.friends_count),
                    "likes"            :int(tweet.favorite_count),
                    "retweets"         :int(tweet.retweet_count),
                    "quotes"           :int(tweet.quote_count),
                    "tweetText"        :tweetText,
                    "sensitive"        :sensitive,
                    "isRetweet"        :tweet.retweeted,
                    "isQuote"          :tweet.is_quote_status
                  }
        try:
            unprocessedArchive.insert_one(item)
        except pymongo.errors.DuplicateKeyError:
            print("duplicate entry found")
            unprocessedArchive.replace_one({'_id': tweetID}, item, upsert=True)
        if(count>=limit):
            return

In [8]:
def loadStockData(stock,tick=False,save=True):
    stocktick = yf.Ticker(stock)
    stockhistory=stocktick.history(period="max")
    if save: dump(stockhistory, stock)
    if tick: return stockhistory,stocktick
    return stockhistory

def getQueryFromMongoDB(tablename,query={},fields=None):
    table=mydb[tablename]
    values=[]
    if fields is not None:
        for entry in table.find(query,fields):
            values.append(entry)
    else :
        for entry in table.find(query):
            values.append(entry)
    return pd.DataFrame(values)

In [10]:
#optimizations 
punctuations=set(string.punctuation)
stopwordset=set(stopwords.words('english'))
trans=str.maketrans('','',string.punctuation)#removes punctuations
#String preprocessing method
    
#Prediction system pipline
def getSentimentNBModel(analyzer="word",ngram=(1,2)):
    sentimentModel = Pipeline([
        ('bow', CountVectorizer(analyzer=analyzer,ngram_range=ngram)),  # strings to token integer counts
        ('tfidf', TfidfTransformer()),  # integer counts to weighted TF-IDF scores
        ('classifier', MultinomialNB()),  # train on TF-IDF vectors w/ Naive Bayes classifier
    ])
    return sentimentModel
def getSentimentLRModel(analyzer="word",ngram=(1,2)):
    sentimentModel = Pipeline([
        ('bow', CountVectorizer(analyzer=analyzer,ngram_range=ngram)),  # strings to token integer counts
        ('tfidf', TfidfTransformer()),  # integer counts to weighted TF-IDF scores
        ('classifier', LogisticRegression()),  # train on TF-IDF vectors w/ Naive Bayes classifier
    ])
    return sentimentModel

In [ ]:
# date="{0}{1}{2}".format(dt.year,dt.month,dt.day)+"0000"
# s="cookie"
# date=  "201701012000"
# todate="201701012315"
# ArchiveStore(q=s, sinceDate=date, untilDate=todate,limit =1)
# dt=(datetime.utcnow() - timedelta(days=1))
# collectDataInterval("cookie")

In [10]:
# file= "project/a/TrainiingNOemoticon.csv"
# df = pd.read_csv(file, header=None)
# for index,row in df.iterrows():
#     x,t_Id,date,c,user,text=row
#     item= {     
#             "SentimentScore"   :x,
#             "_id"              :t_Id,
#             "date"             :date,
#             "poster"           :user,
#             "tweetText"        :text
#           }
#     try:
#         sentiment140.insert_one(item)
#     except pymongo.errors.DuplicateKeyError:
#         print("duplicate entry found")
#         continue
# file= "project/archive/Stocks/tsla.us.txt"
# sentiment140 = mydb["stock inf"]
# df = pd.read_csv(file, header=None,encoding='latin-1')
# for index,row in df.iterrows():
#     x,t_Id,date,c,user,text=row
#     item= {     
#             "Date":,
#             "Open":,
#             "High":,
#             "Low":,
#             "Close":,
#             "Volume":,
#             "OpenInt":
#           }
#     try:
#         sentiment140.insert_one(item)
#     except pymongo.errors.DuplicateKeyError:
#         print("duplicate entry found")
#         continue

# SentimentComponent

In [ ]:
def findMissingInterval(data):
    missingIntervals=[]
    start=None
    end=None
    daysBetween=0
    ss=pd.date_range(data.index.min(), data.index.max())
    for i in range(len(ss)):
        if((ss[i].strftime('%Y-%m-%d')==data.index).sum()>=1):
            if start!=None: 
                missingIntervals.append([start,ss[i],daysBetween])
                start=None
                daysBetween=0
        elif((ss[i].strftime('%Y-%m-%d')==data.index).sum()==0):
            if(start==None): start=ss[i-1]
            daysBetween+=1
    return pd.DataFrame(data=missingIntervals,columns=['From', 'To', 'DaysGap'])
def checkNonEmptyRuns(data):
    runs=[]
    start=None
    end=None
    daysBetween=0
    total=0
    ss=pd.date_range(data.index.min(), data.index.max())
    for i in range(len(ss)):
        if((ss[i].strftime('%Y-%m-%d')==data.index).sum()>=1):
            if(start==None): 
                start=ss[i]
            daysBetween+=1
        elif((ss[i].strftime('%Y-%m-%d')==data.index).sum()==0):
            if start!=None: 
                runs.append([start,ss[i-1],daysBetween])
                start=None
                total+=daysBetween
                daysBetween=0
    if start!=None: 
        runs.append([start,ss[-1],daysBetween])
        total+=daysBetween
    return pd.DataFrame(data=runs,columns=['From', 'To', 'Days'])
def dataSquencer(data,sequenceLength=12,endLength=1):
    xSequences=[]
    ySequences=[]
    sequenceLength+=endLength
    entryList=[]
    ss=pd.date_range(data.index.min(), data.index.max())
    daysBetween=0
    for i in range(len(ss)):
        sss=ss[i].strftime('%Y-%m-%d')
        if((sss==data.index).sum()>=1):
            entryList.append(data.loc[sss].values)
            daysBetween+=1
        elif((sss==data.index).sum()==0):
            if(sequenceLength<=daysBetween):
                xxs,yys=sequenceBreak(entryList,sequenceLength-endLength,endLength)
                xSequences.extend(xxs)
                ySequences.extend(yys)
            entryList=[]
            daysBetween=0
    if(sequenceLength<=daysBetween):
        xxs,yys=sequenceBreak(entryList,sequenceLength-endLength,endLength)
        xSequences.extend(xxs)
        ySequences.extend(yys)
    return xSequences,ySequences
def sequenceBreak(data,sequenceLength=12,endLength=1):
    size=len(data)-sequenceLength-endLength+1
    brokenSequenceXs,brokenSequenceYs=[None]*size,[None]*size
    for i in range(size):
        brokenSequenceXs[i]=data[i:i+sequenceLength]
        brokenSequenceYs[i]=data[sequenceLength+i:i+sequenceLength+endLength]
    return brokenSequenceXs, brokenSequenceYs

def plotCompare(x,y,labels=None,size=(20,10)):
    if labels is None:
        labels=["X,Y"]
    plt.plot(x, label=labels[0])
    plt.plot(y, label=labels[1])
    plt.legend()
    plt.show()

def printResults(y,p,plot=False,size=(20,10)):
    MeanDirectionAccuracy=np.average((np.sign(y[1:] - y[:-1]) == np.sign(p[1:] - p[:-1])))
    RMSE=mean_squared_error(y, p,squared=False)
    print("MSE :",RMSE**2,#Mean Squared Error (MSE)
          "\nRMSE:",RMSE,#Root Mean Square Error
          "\nNMSE:",mean_squared_error(y, p,squared=False)/(max(y)-min(y)),#Normalise Mean Square Error(/max-min)
          "\nDS  :",MeanDirectionAccuracy,#MeanDirectionAccuracy
          "\nWDS :",MeanDirectionAccuracy/(max(y)-min(y)),#Weighted Directional Symmetry
          "\nNs  :",len(y))#Test Samples
    if plot:
        plotCompare(testY,predictions,["actual","Predictions"],size)
        
def makeTrainValidationTestSplit(dataX,dataY=None,splits=[0.6,0.2],randomize=False,reshapeX=None,reshapeY=None,state=0):
    if randomize: dataX,dataY=shuffle(dataX,dataY, random_state=state)
    if reshapeX is not None: dataX=dataX.reshape(reshapeX)
    if reshapeY is not None: dataY=dataY.reshape(reshapeY)
    TLim=round ((dataX.shape[0])*splits[0])
    VLim=round (dataX.shape[0]*splits[1])
    trainX,valX,testX=dataX[:TLim],dataX[TLim:VLim+TLim],dataX[VLim+TLim:]
    if dataY is not None:
        trainY,valY,testY=dataY[:TLim],dataY[TLim:VLim+TLim],dataY[VLim+TLim:]
        return trainX,valX,testX,trainY,valY,testY
    return trainX,valX,testX